In [2]:
# Import all relevant libraries for data analysis and visualization

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from matplotlib.ticker import FuncFormatter

In [3]:
# Import two CSV files entitled "MP_20250716" and "RTD_202507161500"

mp_df = pd.read_csv("MP_20250716.csv")
rtd_df = pd.read_csv("RTD_202507161500.csv")

In [4]:
# Display both imported DataFrames
display(mp_df)
display(rtd_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,COMMODITY_TYPE,MARGINAL_PRICE,Unnamed: 8
0,7/16/2025,RTD,7/16/2025 12:05:00 AM,CMIN,14SARANG_U01,G,En,3037.0,NaN
1,7/16/2025 12:05:00 AM,RTD,7/16/2025 12:10:00 AM,CMIN,10GNPK_U04,G,En,3348.3,NaN
2,7/16/2025 12:10:00 AM,RTD,7/16/2025 12:15:00 AM,CLUZ,03STA-RI_G02,G,En,5357.5,NaN
3,7/16/2025 12:10:00 AM,RTD,7/16/2025 12:15:00 AM,CLUZ,03STA-RI_G03,G,En,5357.5,NaN
4,7/16/2025 12:10:00 AM,RTD,7/16/2025 12:15:00 AM,CLUZ,03STA-RI_G04,G,En,5357.5,NaN
...,...,...,...,...,...,...,...,...,...
890,7/16/2025 11:50:00 PM,RTD,7/16/2025 11:55:00 PM,CMIN,14MTAPO_U02,G,En,0.0,NaN
891,7/16/2025 11:55:00 PM,RTD,7/17/2025,CVIS,08PEDC_U01,G,En,3130.0,NaN
892,7/16/2025 11:55:00 PM,RTD,7/17/2025,CVIS,08PEDC_U02,G,En,3130.0,NaN
893,7/16/2025 11:55:00 PM,RTD,7/17/2025,CMIN,11FDC_U02,G,En,2159.8,NaN


,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION
0,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01ACNPC_G01,G,0.00,3640.0090,0.9881,3596.4914,43.5175,0.0
1,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AEC_L01,NL,-60.68,3645.7633,0.9865,3596.4914,49.2719,0.0
2,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_SS,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
3,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_T1L1,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
4,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_U01,G,37.50,3655.8335,0.9837,3596.4914,59.3421,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13364,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TACSUR_L02,NL,-29.82,3912.0749,0.9182,3592.3553,319.7196,0.0
13365,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TACUR_SS,NL,0.00,3912.0749,0.9182,3592.3553,319.7196,0.0
13366,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TIN_T1L1,NL,-14.91,3791.3718,0.9475,3592.3553,199.0165,0.0
13367,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TIN_T1L2,NL,-102.57,3791.3718,0.9475,3592.3553,199.0165,0.0


In [5]:
# Show the column names for both DataFrames
print("mp_df columns:")
print(mp_df.columns.tolist())

print("\nrtd_df columns:")
print(rtd_df.columns.tolist())

mp_df columns:
['RUN_TIME', 'MKT_TYPE', 'TIME_INTERVAL', 'REGION_NAME', 'RESOURCE_NAME', 'RESOURCE_TYPE', 'COMMODITY_TYPE', 'MARGINAL_PRICE', 'Unnamed: 8']

rtd_df columns:
['RUN_TIME', 'MKT_TYPE', 'TIME_INTERVAL', 'REGION_NAME', 'RESOURCE_NAME', 'RESOURCE_TYPE', 'SCHED_MW', 'LMP', 'LOSS_FACTOR', 'LMP_SMP', 'LMP_LOSS', 'LMP_CONGESTION']


In [6]:
# Convert RUN_TIME and TIME_INTERVAL in mp_df to military time (24-hour format)
# If RUN_TIME is just a date (e.g. '7/16/2025'), set it to '00:00' for that day
def fix_run_time(val, prev_time=None):
    try:
        dt = pd.to_datetime(val, errors='coerce')
        if pd.isnull(dt):
            return np.nan
        # If original string has no time, set to 00:00
        if len(str(val).strip()) <= 10:
            return dt.strftime('%Y-%m-%d 00:00')
        else:
            return dt.strftime('%Y-%m-%d %H:%M')
    except:
        return np.nan

mp_df['RUN_TIME'] = [fix_run_time(val) for val in mp_df['RUN_TIME']]
mp_df['TIME_INTERVAL'] = pd.to_datetime(mp_df['TIME_INTERVAL'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M')

# Display to verify conversion
print(mp_df[['RUN_TIME', 'TIME_INTERVAL']].head())

           RUN_TIME     TIME_INTERVAL
0  2025-07-16 00:00  2025-07-16 00:05
1  2025-07-16 00:05  2025-07-16 00:10
2  2025-07-16 00:10  2025-07-16 00:15
3  2025-07-16 00:10  2025-07-16 00:15
4  2025-07-16 00:10  2025-07-16 00:15


C:\Users\theam\AppData\Local\Temp\ipykernel_12616\427894902.py:17: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  mp_df['TIME_INTERVAL'] = pd.to_datetime(mp_df['TIME_INTERVAL'], errors='coerce').dt.strftime('%Y-%m-%d %H:%M')


In [7]:
# Display both imported DataFrames
display(mp_df)
display(rtd_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,COMMODITY_TYPE,MARGINAL_PRICE,Unnamed: 8
0,2025-07-16 00:00,RTD,2025-07-16 00:05,CMIN,14SARANG_U01,G,En,3037.0,NaN
1,2025-07-16 00:05,RTD,2025-07-16 00:10,CMIN,10GNPK_U04,G,En,3348.3,NaN
2,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G02,G,En,5357.5,NaN
3,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G03,G,En,5357.5,NaN
4,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G04,G,En,5357.5,NaN
...,...,...,...,...,...,...,...,...,...
890,2025-07-16 23:50,RTD,2025-07-16 23:55,CMIN,14MTAPO_U02,G,En,0.0,NaN
891,2025-07-16 23:55,RTD,2025-07-17 00:00,CVIS,08PEDC_U01,G,En,3130.0,NaN
892,2025-07-16 23:55,RTD,2025-07-17 00:00,CVIS,08PEDC_U02,G,En,3130.0,NaN
893,2025-07-16 23:55,RTD,2025-07-17 00:00,CMIN,11FDC_U02,G,En,2159.8,NaN


,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION
0,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01ACNPC_G01,G,0.00,3640.0090,0.9881,3596.4914,43.5175,0.0
1,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AEC_L01,NL,-60.68,3645.7633,0.9865,3596.4914,49.2719,0.0
2,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_SS,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
3,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_T1L1,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
4,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_U01,G,37.50,3655.8335,0.9837,3596.4914,59.3421,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13364,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TACSUR_L02,NL,-29.82,3912.0749,0.9182,3592.3553,319.7196,0.0
13365,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TACUR_SS,NL,0.00,3912.0749,0.9182,3592.3553,319.7196,0.0
13366,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TIN_T1L1,NL,-14.91,3791.3718,0.9475,3592.3553,199.0165,0.0
13367,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TIN_T1L2,NL,-102.57,3791.3718,0.9475,3592.3553,199.0165,0.0


In [8]:
# 1. Remove rows with any NaN in key columns (RUN_TIME, TIME_INTERVAL)
mp_df = mp_df.dropna(subset=['RUN_TIME', 'TIME_INTERVAL'])
rtd_df = rtd_df.dropna(subset=['RUN_TIME', 'TIME_INTERVAL'])

# 2. Remove rows with placeholder or irrelevant values (e.g., 'EOF')
mp_df = mp_df[~mp_df['RUN_TIME'].str.contains('EOF', na=False)]
rtd_df = rtd_df[~rtd_df['RUN_TIME'].str.contains('EOF', na=False)]

# 3. Drop completely empty columns (like 'Unnamed: 8' in mp_df)
mp_df = mp_df.drop(columns=[col for col in mp_df.columns if 'Unnamed' in col], errors='ignore')

# 4. Remove duplicate rows if any
mp_df = mp_df.drop_duplicates()
rtd_df = rtd_df.drop_duplicates()

# 5. Reset index for convenience
mp_df = mp_df.reset_index(drop=True)
rtd_df = rtd_df.reset_index(drop=True)

# 6. (Optional) Convert numeric columns to proper types
if 'MARGINAL_PRICE' in mp_df.columns:
    mp_df['MARGINAL_PRICE'] = pd.to_numeric(mp_df['MARGINAL_PRICE'], errors='coerce')
if 'SCHED_MW' in rtd_df.columns:
    rtd_df['SCHED_MW'] = pd.to_numeric(rtd_df['SCHED_MW'], errors='coerce')

# 7. (Optional) Standardize column names to avoid merge issues
mp_df.columns = mp_df.columns.str.strip()
rtd_df.columns = rtd_df.columns.str.strip()

# Display cleaned DataFrames
display(mp_df)
display(rtd_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,COMMODITY_TYPE,MARGINAL_PRICE
0,2025-07-16 00:00,RTD,2025-07-16 00:05,CMIN,14SARANG_U01,G,En,3037.0
1,2025-07-16 00:05,RTD,2025-07-16 00:10,CMIN,10GNPK_U04,G,En,3348.3
2,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G02,G,En,5357.5
3,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G03,G,En,5357.5
4,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G04,G,En,5357.5
...,...,...,...,...,...,...,...,...
889,2025-07-16 23:50,RTD,2025-07-16 23:55,CMIN,14MTAPO_U01,G,En,0.0
890,2025-07-16 23:50,RTD,2025-07-16 23:55,CMIN,14MTAPO_U02,G,En,0.0
891,2025-07-16 23:55,RTD,2025-07-17 00:00,CVIS,08PEDC_U01,G,En,3130.0
892,2025-07-16 23:55,RTD,2025-07-17 00:00,CVIS,08PEDC_U02,G,En,3130.0


,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION
0,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01ACNPC_G01,G,0.00,3640.0090,0.9881,3596.4914,43.5175,0.0
1,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AEC_L01,NL,-60.68,3645.7633,0.9865,3596.4914,49.2719,0.0
2,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_SS,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
3,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_T1L1,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
4,7/16/2025 14:00,RTD,7/16/2025 14:05,CLUZ,01AMBUK_U01,G,37.50,3655.8335,0.9837,3596.4914,59.3421,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13363,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14SUPKOR_G01,G,0.00,3912.0749,0.9183,3592.3553,319.7196,0.0
13364,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TACSUR_L02,NL,-29.82,3912.0749,0.9182,3592.3553,319.7196,0.0
13365,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TACUR_SS,NL,0.00,3912.0749,0.9182,3592.3553,319.7196,0.0
13366,7/16/2025 14:55,RTD,7/16/2025 15:00,CMIN,14TIN_T1L1,NL,-14.91,3791.3718,0.9475,3592.3553,199.0165,0.0


In [9]:
# Standardize RUN_TIME and TIME_INTERVAL to military time in both DataFrames
for col in ['RUN_TIME', 'TIME_INTERVAL']:
    mp_df[col] = pd.to_datetime(mp_df[col], errors='coerce').dt.strftime('%Y-%m-%d %H:%M')
    rtd_df[col] = pd.to_datetime(rtd_df[col], errors='coerce').dt.strftime('%Y-%m-%d %H:%M')

display(mp_df)
display(rtd_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,COMMODITY_TYPE,MARGINAL_PRICE
0,2025-07-16 00:00,RTD,2025-07-16 00:05,CMIN,14SARANG_U01,G,En,3037.0
1,2025-07-16 00:05,RTD,2025-07-16 00:10,CMIN,10GNPK_U04,G,En,3348.3
2,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G02,G,En,5357.5
3,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G03,G,En,5357.5
4,2025-07-16 00:10,RTD,2025-07-16 00:15,CLUZ,03STA-RI_G04,G,En,5357.5
...,...,...,...,...,...,...,...,...
889,2025-07-16 23:50,RTD,2025-07-16 23:55,CMIN,14MTAPO_U01,G,En,0.0
890,2025-07-16 23:50,RTD,2025-07-16 23:55,CMIN,14MTAPO_U02,G,En,0.0
891,2025-07-16 23:55,RTD,2025-07-17 00:00,CVIS,08PEDC_U01,G,En,3130.0
892,2025-07-16 23:55,RTD,2025-07-17 00:00,CVIS,08PEDC_U02,G,En,3130.0


,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION
0,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01ACNPC_G01,G,0.00,3640.0090,0.9881,3596.4914,43.5175,0.0
1,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AEC_L01,NL,-60.68,3645.7633,0.9865,3596.4914,49.2719,0.0
2,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_SS,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
3,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_T1L1,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0
4,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U01,G,37.50,3655.8335,0.9837,3596.4914,59.3421,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
13363,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SUPKOR_G01,G,0.00,3912.0749,0.9183,3592.3553,319.7196,0.0
13364,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14TACSUR_L02,NL,-29.82,3912.0749,0.9182,3592.3553,319.7196,0.0
13365,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14TACUR_SS,NL,0.00,3912.0749,0.9182,3592.3553,319.7196,0.0
13366,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14TIN_T1L1,NL,-14.91,3791.3718,0.9475,3592.3553,199.0165,0.0


In [10]:
# Merge on RUN_TIME, TIME_INTERVAL, and REGION_NAME
merged_df = pd.merge(
    rtd_df,
    mp_df[['RUN_TIME', 'TIME_INTERVAL', 'REGION_NAME', 'MARGINAL_PRICE']],
    on=['RUN_TIME', 'TIME_INTERVAL', 'REGION_NAME'],
    how='left'
)

display(merged_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION,MARGINAL_PRICE
0,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01ACNPC_G01,G,0.00,3640.0090,0.9881,3596.4914,43.5175,0.0,3536.4300
1,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AEC_L01,NL,-60.68,3645.7633,0.9865,3596.4914,49.2719,0.0,3536.4300
2,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_SS,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0,3536.4300
3,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_T1L1,NL,0.00,3657.6318,0.9833,3596.4914,61.1404,0.0,3536.4300
4,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U01,G,37.50,3655.8335,0.9837,3596.4914,59.3421,0.0,3536.4300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18676,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14TACUR_SS,NL,0.00,3912.0749,0.9182,3592.3553,319.7196,0.0,3458.0012
18677,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14TIN_T1L1,NL,-14.91,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012
18678,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14TIN_T1L1,NL,-14.91,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012
18679,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14TIN_T1L2,NL,-102.57,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012


In [11]:
# Filter merged_df to keep only rows where SCHED_MW > 0
filtered_df = merged_df[merged_df['SCHED_MW'] > 0].reset_index(drop=True)

display(filtered_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION,MARGINAL_PRICE
0,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U01,G,37.5,3655.8335,0.9837,3596.4914,59.3421,0.0,3536.4300
1,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U02,G,20.5,3656.5528,0.9836,3596.4914,60.0614,0.0,3536.4300
2,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U03,G,37.5,3655.8335,0.9838,3596.4914,59.3421,0.0,3536.4300
3,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMPHAW_G01,G,8.9,3656.1932,0.9837,3596.4914,59.7018,0.0,3536.4300
4,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01ANDA_G01,G,65.5,3645.7633,0.9865,3596.4914,49.2719,0.0,3536.4300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4407,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SARANG_U02,G,117.0,3681.8049,0.9757,3592.3553,89.4496,0.0,3458.0012
4408,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SIGHYDRO_G01,G,11.0,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012
4409,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SIGHYDRO_G01,G,11.0,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012
4410,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SPGI_U01,G,5.2,3912.0749,0.9183,3592.3553,319.7196,0.0,3458.0012


In [12]:
# Remove rows where MARGINAL_PRICE is empty (NaN)
filtered_df = filtered_df.dropna(subset=['MARGINAL_PRICE']).reset_index(drop=True)

display(filtered_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION,MARGINAL_PRICE
0,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U01,G,37.5,3655.8335,0.9837,3596.4914,59.3421,0.0,3536.4300
1,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U02,G,20.5,3656.5528,0.9836,3596.4914,60.0614,0.0,3536.4300
2,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U03,G,37.5,3655.8335,0.9838,3596.4914,59.3421,0.0,3536.4300
3,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMPHAW_G01,G,8.9,3656.1932,0.9837,3596.4914,59.7018,0.0,3536.4300
4,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01ANDA_G01,G,65.5,3645.7633,0.9865,3596.4914,49.2719,0.0,3536.4300
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3862,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SARANG_U02,G,117.0,3681.8049,0.9757,3592.3553,89.4496,0.0,3458.0012
3863,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SIGHYDRO_G01,G,11.0,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012
3864,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SIGHYDRO_G01,G,11.0,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012
3865,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SPGI_U01,G,5.2,3912.0749,0.9183,3592.3553,319.7196,0.0,3458.0012


In [13]:
# List all unique RESOURCE_NAME values in filtered_df (no duplicates in the list)
unique_resource_names = filtered_df['RESOURCE_NAME'].unique()

print("All unique RESOURCE_NAME values:")
for name in unique_resource_names:
    print(name)

All unique RESOURCE_NAME values:
01AMBUK_U01
01AMBUK_U02
01AMBUK_U03
01AMPHAW_G01
01ANDA_G01
01ANGAT_A
01ANGAT_M
01APEC_G01
01ARAYSOL_G01
01ARAYSOL_G02
01ARESOL_G01
01ARMSOL_G01
01BAKSIP_G01
01BAKUN_G01
01BALWIND_G01
01BINENG_G01
01BINGA_U01
01BINGA_U02
01BINGA_U03
01BINGA_U04
01BONGSOL_G01
01BOSUNG_G01
01BT2020_G01
01BTNSOL_G01
01BTSOLEN_G01
01BULSOL_G01
01BURGOS_G01
01BURGOS_G02
01BURGOS_G03
01CABSOL_G01
01CAGBIO_G01
01CAGYSOL_G01
01CAPRIS_G01
01CAPRIS_G02
01CASECN_U01
01CAYBSOL_G01
01CLASOL_G01
01CONSOL_G01
01CORDONSOL_G01
01DALSOL_G01
01DOMSOL_G01
01GFII_G01
01GIFT_G01
01GIFT_G02
01GIGSOL_G01
01GNPD_U01
01GNPD_U02
01GRGOLD_G01
01IBEC_G01
01IBULAO_G01
01IPOWER_G02
01LAOSOL_G01
01LASUER_U01
01MAGAT_U01
01MAGAT_U02
01MAGAT_U04
01MANSOK_G01
01MARIS_U01
01MARIS_U02
01MARSOL_G01
01MARVEL_G01
01MASIWA_G01
01MATUNO_G01
01MEGASOL_G01
01MPGC_U02
01MPGC_U03
01MPGC_U04
01MSINLO_G01
01MSINLO_G02
01MSINLO_G03
01NIABAL_G01
01NMHC_G01
01NMHC_G03
01NUEVASOL_G01
01NWIND_G01
01NWIND_G02
01PASQSOL_G01

In [14]:
# Dictionary for RESOURCE_NAME to technology/fuel type

tech_dict = {
    "01AMBUK_U01": "Hydro",
    "01AMBUK_U02": "Hydro",
    "01AMBUK_U03": "Hydro",
    "01AMPHAW_G01": "Hydro",
    "01ANDA_G01": "Coal",
    "01ANGAT_A": "Hydro",
    "01ANGAT_M": "Hydro",
    "01APEC_G01": "Coal",
    "01ARAYSOL_G01": "Solar",
    "01ARAYSOL_G02": "Solar",
    "01ARESOL_G01": "Solar",
    "01ARMSOL_G01": "Solar",
    "01BAKSIP_G01": "Hydro",
    "01BAKUN_G01": "Hydro",
    "01BALWIND_G01": "Wind",
    "01BINENG_G01": "Hydro",
    "01BINGA_U01": "Hydro",
    "01BINGA_U02": "Hydro",
    "01BINGA_U03": "Hydro",
    "01BINGA_U04": "Hydro",
    "01BONGSOL_G01": "Solar",
    "01BOSUNG_G01": "Solar",
    "01BT2020_G01": "Biomass",
    "01BTNSOL_G01": "Solar",
    "01BTSOLEN_G01": "Solar",
    "01BULSOL_G01": "Solar",
    "01BURGOS_G01": "Solar",
    "01BURGOS_G02": "Solar",
    "01BURGOS_G03": "Solar",
    "01CABSOL_G01": "Solar",
    "01CAGBIO_G01": "Biomass",
    "01CAGYSOL_G01": "Solar",
    "01CAPRIS_G01": "Wind",
    "01CAPRIS_G02": "Wind",
    "01CASECN_U01": "Hydro",
    "01CAYBSOL_G01": "Solar",
    "01CLASOL_G01": "Solar",
    "01CONSOL_G01": "Solar",
    "01CORDONSOL_G01": "Solar",
    "01DALSOL_G01": "Solar",
    "01DOMSOL_G01": "Solar",
    "01GFII_G01": "Biomass",
    "01GIFT_G01": "Biomass",
    "01GIFT_G02": "Biomass",
    "01GIGSOL_G01": "Solar",
    "01GNPD_U01": "Coal",
    "01GNPD_U02": "Coal",
    "01GRGOLD_G01": "Biomass",
    "01IBEC_G01": "Biomass",
    "01IBULAO_G01": "Hydro",
    "01IPOWER_G02": "Biomass",
    "01LAOSOL_G01": "Solar",
    "01LASUER_U01": "Biomass",
    "01MAGAT_U01": "Hydro",
    "01MAGAT_U02": "Hydro",
    "01MAGAT_U04": "Hydro",
    "01MANSOK_G01": "Unknown",
    "01MARIS_U01": "Hydro",
    "01MARIS_U02": "Hydro",
    "01MARSOL_G01": "Solar",
    "01MARVEL_G01": "Coal",
    "01MASIWA_G01": "Hydro",
    "01MATUNO_G01": "Hydro",
    "01MEGASOL_G01": "Solar",
    "01MPGC_U02": "Coal",
    "01MPGC_U03": "Coal",
    "01MPGC_U04": "Coal",
    "01MSINLO_G01": "Coal",
    "01MSINLO_G02": "Coal",
    "01MSINLO_G03": "Coal",
    "01NIABAL_G01": "Hydro",
    "01NMHC_G01": "Hydro",
    "01NMHC_G03": "Hydro",
    "01NUEVASOL_G01": "Solar",
    "01NWIND_G01": "Wind",
    "01NWIND_G02": "Wind",
    "01PASQSOL_G01": "Solar",
    "01PAVGSOL_G01": "Solar",
    "01PETRON_G01": "Coal",
    "01PETSOL_G01": "Solar",
    "01PETSOL_G02": "Solar",
    "01PNTBNG_U02": "Hydro",
    "01RASLAG_G01": "Solar",
    "01RASLAG_G02": "Solar",
    "01RASLAG_G03": "Solar",
    "01RASLAG_G04": "Solar",
    "01SABANG_G01": "Hydro",
    "01SAMLSOL_G01": "Solar",
    "01SANMIGSOL_G01": "Solar",
    "01SHIZEN_G01": "Solar",
    "01SJSOL_G01": "Solar",
    "01SLANGN_G01": "Hydro",
    "01SMBELL_G01": "Hydro",
    "01SMC_G01": "Coal",
    "01SMC_G02": "Coal",
    "01SMC_G03": "Coal",
    "01SMC_G04": "Coal",
    "01SNMARSOL_G01": "Solar",
    "01SPABUL_G01": "Solar",
    "01SROQUE_U02": "Hydro",
    "01SROQUE_U03": "Hydro",
    "01SUAL_G01": "Coal",
    "01SUAL_G02": "Coal",
    "01SUBSOL_G01": "Solar",
    "01SUPSOL_G01": "Solar",
    "01TERASU_G01": "Solar",
    "01TRUSTSOL_G01": "Solar",
    "01VSGRIP_G01": "Biomass",
    "01YHGRN_G01": "Solar",
    "01ZAMSOL_G01": "Solar",
    "02ECOPRK_G01": "Solar",
    "02ECOTAGA_G01": "Solar",
    "02MMPP_G01": "Biomass",
    "02PNGEA_G01": "Biomass",
    "02PNGYSOL_G01": "Solar",
    "02VALSOL_G01": "Solar",
    "03ADISOL_G01": "Solar",
    "03AWOC_G01": "Wind",
    "03BACMAN_BAT": "Battery",
    "03BACMAN_U01": "Geothermal",
    "03BACMAN_U03": "Geothermal",
    "03BALUG_G01": "Hydro",
    "03BART_G01": "Hydro",
    "03BBEC_G01": "Biomass",
    "03BOTOCA_G01": "Hydro",
    "03CALACA_G01": "Coal",
    "03CALACA_G02": "Coal",
    "03CALIRY_G01": "Hydro",
    "03CALSOL_G01": "Solar",
    "03CLABSOL_G01": "Solar",
    "03CLBATO_G01": "Hydro",
    "03EERI_G01": "Gas",
    "03EERI_G02": "Gas",
    "03EERI_G03": "Gas",
    "03ILIJAN_G01": "Gas",
    "03ILIJAN_G02": "Gas",
    "03INARI_G01": "Hydro",
    "03KAL_G03": "Hydro",
    "03KAL_G04": "Hydro",
    "03LWERLAB_G01": "Hydro",
    "03MAJAY_G01": "Hydro",
    "03MARAGSOL_G01": "Solar",
    "03MEC_G01": "Solar",
    "03MGI_G02": "Geothermal",
    "03MGPP_G01": "Geothermal",
    "03MKBN_A": "Geothermal",
    "03MKBN_B": "Geothermal",
    "03MKBN_D": "Geothermal",
    "03MKBN_E": "Geothermal",
    "03NAICSOL_G01": "Solar",
    "03ORMAT_G01": "Geothermal",
    "03PAGBIL_G02": "Coal",
    "03PALAK_G01": "Hydro",
    "03PALAYAN_G01": "Geothermal",
    "03RCBMI_G02": "Diesel",
    "03SBPL_G01": "Coal",
    "03SLPGC_G01": "Coal",
    "03SLPGC_G02": "Coal",
    "03SLTEC_G02": "Coal",
    "03SNGAB_G01": "Gas",
    "03SOLACE_G01": "Solar",
    "03STA-RI_G01": "Gas",
    "03STA-RI_G02": "Gas",
    "03STA-RI_G03": "Gas",
    "03STA-RI_G04": "Gas",
    "03STA-RI_G05": "Gas",
    "03STA-RI_G06": "Gas",
    "03TANSOL_G01": "Solar",
    "03TIBAG_G01": "Hydro",
    "03TIWI_A": "Geothermal",
    "03TIWI_C": "Geothermal",
    "03UPLAB_G01": "Hydro",
    "09LIBPOWR_G01": "Biomass",
    "10AGUS1_U01": "Hydro",
    "10AGUS2_U01": "Hydro",
    "10AGUS2_U02": "Hydro",
    "10AGUS2_U03": "Hydro",
    "10AGUS4_U01": "Hydro",
    "10AGUS4_U02": "Hydro",
    "10AGUS4_U03": "Hydro",
    "10AGUS5_U02": "Hydro",
    "10AGUS6_U01": "Hydro",
    "10AGUS6_U05": "Hydro",
    "10AGUS7_U01": "Hydro",
    "10AGUS7_U02": "Hydro",
    "10GNPK_U01": "Coal",
    "10GNPK_U02": "Coal",
    "10GNPK_U03": "Coal",
    "10GNPK_U04": "Coal",
    "10LIAN_G01": "Hydro",
    "11FDC_U01": "Coal",
    "11FDC_U02": "Coal",
    "11FDC_U03": "Coal",
    "11FGBPC_G01": "Hydro",
    "11KBSOL_G01": "Solar",
    "11KIRAS_G01": "Solar",
    "11MANFOR_G01": "Hydro",
    "11MANFOR_G02": "Hydro",
    "11MINBAL_G01": "Unknown",
    "11MINBU_G01": "Hydro",
    "11MNCBLG_G01": "Hydro",
    "11PULA4_U02": "Hydro",
    "11PULA4_U03": "Hydro",
    "12ASIGA_G01": "Hydro",
    "12LKMAINIT_G01": "Hydro",
    "12TM2_U02": "Diesel",
    "13DCPP_U01": "Coal",
    "13DCPP_U02": "Coal",
    "13DIGSOL_G01": "Solar",
    "13EUROH_G01": "Hydro",
    "13SIBULAN_G01": "Hydro",
    "13SMC_U01": "Coal",
    "13TALOM1_G01": "Hydro",
    "13TALOM23_G01": "Hydro",
    "13TM1_U02": "Diesel",
    "13TUDAY2_G01": "Hydro",
    "14ASTROSOL_G01": "Solar",
    "14BIOGS_G01": "BioGas",
    "14MARBEL_U01": "Hydro",
    "14MTAPO_U01": "Geothermal",
    "14MTAPO_U02": "Geothermal",
    "14MTAPO_U03": "Geothermal",
    "14NVSOL_G01": "Solar",
    "14SARANG_U01": "Coal",
    "14SARANG_U02": "Coal",
    "14SIGHYDRO_G01": "Hydro",
    "14SPGI_U01": "Biomass",
    "04IASMOD_G05": "Diesel",
    "04LEYTE_A": "Geothermal",
    "04LGPP_G01": "Geothermal",
    "04PHSOL_G01": "Solar",
    "04SEPSOL_G01": "Solar",
    "04TAFT_G01": "Hydro",
    "04TONGO_BAT": "Battery",
    "04UTH_G01": "Hydro",
    "05CEDC_U02": "Coal",
    "05CEDC_U03": "Coal",
    "05EAUC_U02": "Diesel",
    "05EAUC_U03": "Diesel",
    "05EAUC_U04": "Diesel",
    "05KSPC_G01": "Coal",
    "05KSPC_G02": "Coal",
    "05THVI_U01": "Coal",
    "05TOLSOL_G01": "Solar",
    "05TPC_G02": "Coal",
    "06AMLA_G01": "Hydro",
    "06CABI_G01": "Biomass",
    "06CALASOL_G01": "Solar",
    "06CARSOL_G01": "Solar",
    "06CENPRI_U01": "Diesel",
    "06CENPRI_U02": "Diesel",
    "06CENPRI_U03": "Diesel",
    "06CENPRI_U05": "Diesel",
    "06HELIOS_G01": "Solar",
    "06MANSOL_G01": "Solar",
    "06MNTSOL_G01": "Solar",
    "06PAL1A_G01": "Geothermal",
    "06PAL2A_U01": "Geothermal",
    "06PAL2A_U02": "Geothermal",
    "06PAL2A_U03": "Geothermal",
    "06SACASL_G01": "Solar",
    "06SACASL_G02": "Solar",
    "06SACSUN_G01": "Solar",
    "06SLYSOL_G01": "Solar",
    "06URC_G01": "Biomass",
    "07BIDPP_G01": "Unknown",
    "07DAGSOL_G01": "Solar",
    "07LOBOC_G01": "Hydro",
    "07LOBOC_G02": "Hydro",
    "07SEVILL_G01": "Hydro",
    "08PALM_G01": "Coal",
    "08PEDC_U01": "Coal",
    "08PEDC_U02": "Coal",
    "08PEDC_U03": "Coal",
    "08PWIND_G01": "Wind",
    "08PWIND_G02": "Wind",
    "08SLWIND_G01": "Wind",
    "08STBPB1_U01": "Diesel",
    "08STBPB1_U02": "Diesel",
    "08STBPB1_U04": "Diesel",
    "08SUWECO_G01": "Hydro",
    "08TIMBA_G01": "Hydro",
    "03TANAWON_G01": "Unknown"
}

In [15]:
# Add a new column 'TECHNOLOGY' to filtered_df using the tech_dict mapping
filtered_df['TECHNOLOGY'] = filtered_df['RESOURCE_NAME'].map(tech_dict)

# Show RESOURCE_NAME and TECHNOLOGY side by side
display(filtered_df[['RESOURCE_NAME', 'TECHNOLOGY']])

,RESOURCE_NAME,TECHNOLOGY
0,01AMBUK_U01,Hydro
1,01AMBUK_U02,Hydro
2,01AMBUK_U03,Hydro
3,01AMPHAW_G01,Hydro
4,01ANDA_G01,Coal
...,...,...
3862,14SARANG_U02,Coal
3863,14SIGHYDRO_G01,Hydro
3864,14SIGHYDRO_G01,Hydro
3865,14SPGI_U01,Biomass


In [16]:
display(filtered_df )

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION,MARGINAL_PRICE,TECHNOLOGY
0,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U01,G,37.5,3655.8335,0.9837,3596.4914,59.3421,0.0,3536.4300,Hydro
1,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U02,G,20.5,3656.5528,0.9836,3596.4914,60.0614,0.0,3536.4300,Hydro
2,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMBUK_U03,G,37.5,3655.8335,0.9838,3596.4914,59.3421,0.0,3536.4300,Hydro
3,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01AMPHAW_G01,G,8.9,3656.1932,0.9837,3596.4914,59.7018,0.0,3536.4300,Hydro
4,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01ANDA_G01,G,65.5,3645.7633,0.9865,3596.4914,49.2719,0.0,3536.4300,Coal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3862,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SARANG_U02,G,117.0,3681.8049,0.9757,3592.3553,89.4496,0.0,3458.0012,Coal
3863,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SIGHYDRO_G01,G,11.0,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012,Hydro
3864,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SIGHYDRO_G01,G,11.0,3791.3718,0.9475,3592.3553,199.0165,0.0,3458.0012,Hydro
3865,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,14SPGI_U01,G,5.2,3912.0749,0.9183,3592.3553,319.7196,0.0,3458.0012,Biomass


In [17]:
# List all unique TECHNOLOGY values in filtered_df
all_technologies = filtered_df['TECHNOLOGY'].dropna().unique()
print("All TECHNOLOGY values:")
for tech in all_technologies:
    print(tech)

All TECHNOLOGY values:
Hydro
Coal
Solar
Wind
Biomass
Unknown
Battery
Geothermal
Gas
Diesel
BioGas


In [18]:
re_dict = {
    "Hydro": "Renewable",
    "Coal": "Non-Renewable",
    "Solar": "Renewable",
    "Wind": "Renewable",
    "Biomass": "Renewable",
    "Unknown": "Unknown",
    "Battery": "Non-Renewable",
    "Geothermal": "Renewable",
    "Gas": "Non-Renewable",
    "Diesel": "Non-Renewable",
    "BioGas": "Renewable",
}

In [19]:
filtered_df['RENEWABLE'] = filtered_df['TECHNOLOGY'].map(re_dict)

# Display to verify
display(filtered_df[['RESOURCE_NAME', 'TECHNOLOGY', 'RENEWABLE']])

,RESOURCE_NAME,TECHNOLOGY,RENEWABLE
0,01AMBUK_U01,Hydro,Renewable
1,01AMBUK_U02,Hydro,Renewable
2,01AMBUK_U03,Hydro,Renewable
3,01AMPHAW_G01,Hydro,Renewable
4,01ANDA_G01,Coal,Non-Renewable
...,...,...,...
3862,14SARANG_U02,Coal,Non-Renewable
3863,14SIGHYDRO_G01,Hydro,Renewable
3864,14SIGHYDRO_G01,Hydro,Renewable
3865,14SPGI_U01,Biomass,Renewable


In [20]:
# Regression analysis for Renewable technologies only: MARGINAL_PRICE ~ SCHED_MW

renewable_df = filtered_df[filtered_df['RENEWABLE'] == 'Renewable'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(renewable_df) >= 2:
    X = renewable_df['SCHED_MW']
    y = renewable_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    renewable_model = sm.OLS(y, X).fit()
    print(renewable_model.summary())
else:
    print("Not enough data for Renewable regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     6.311
Date:                Tue, 09 Sep 2025   Prob (F-statistic):             0.0121
Time:                        14:16:23   Log-Likelihood:                -27587.
No. Observations:                2816   AIC:                         5.518e+04
Df Residuals:                    2814   BIC:                         5.519e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5884.2897     99.003     59.436      0.0

In [21]:
# Regression analysis for Hydro only: MARGINAL_PRICE ~ SCHED_MW

# Filter for Hydro technology
hydro_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Hydro'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(hydro_df) >= 2:
    X = hydro_df['SCHED_MW']
    y = hydro_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    hydro_model = sm.OLS(y, X).fit()
    print(hydro_model.summary())
else:
    print("Not enough data for Hydro regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.031
Method:                 Least Squares   F-statistic:                     40.45
Date:                Tue, 09 Sep 2025   Prob (F-statistic):           2.82e-10
Time:                        14:16:23   Log-Likelihood:                -11991.
No. Observations:                1254   AIC:                         2.399e+04
Df Residuals:                    1252   BIC:                         2.400e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5661.2215    123.750     45.747      0.0

In [22]:
from statsmodels.stats.stattools import durbin_watson

# Example for Hydro regression
if len(hydro_df) >= 2:
    dw_stat = durbin_watson(hydro_model.resid)
    print(f"Durbin-Watson statistic for Hydro regression: {dw_stat}")

Durbin-Watson statistic for Hydro regression: 0.13525835893354393


In [23]:
# Regression analysis for Solar only: MARGINAL_PRICE ~ SCHED_MW

# Filter for Solar technology
solar_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Solar'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(solar_df) >= 2:
    X = solar_df['SCHED_MW']
    y = solar_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    solar_model = sm.OLS(y, X).fit()
    print(solar_model.summary())
else:
    print("Not enough data for Solar regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     3.731
Date:                Tue, 09 Sep 2025   Prob (F-statistic):             0.0537
Time:                        14:16:23   Log-Likelihood:                -9232.8
No. Observations:                 937   AIC:                         1.847e+04
Df Residuals:                     935   BIC:                         1.848e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6790.2905    199.236     34.082      0.0

In [24]:
# Regression analysis for Coal only: MARGINAL_PRICE ~ SCHED_MW

# Filter for Coal technology
coal_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Coal'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(coal_df) >= 2:
    X = coal_df['SCHED_MW']
    y = coal_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    coal_model = sm.OLS(y, X).fit()
    print(coal_model.summary())
else:
    print("Not enough data for Coal regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.047
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     33.08
Date:                Tue, 09 Sep 2025   Prob (F-statistic):           1.34e-08
Time:                        14:16:23   Log-Likelihood:                -6666.7
No. Observations:                 674   AIC:                         1.334e+04
Df Residuals:                     672   BIC:                         1.335e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       7873.9045    312.738     25.177      0.0

In [25]:
# Filter for Wind technology
wind_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Wind'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(wind_df) >= 2:
    X = wind_df['SCHED_MW']
    y = wind_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    wind_model = sm.OLS(y, X).fit()
    print(wind_model.summary())
else:
    print("Not enough data for Wind regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     7.947
Date:                Tue, 09 Sep 2025   Prob (F-statistic):            0.00578
Time:                        14:16:23   Log-Likelihood:                -1056.5
No. Observations:                 105   AIC:                             2117.
Df Residuals:                     103   BIC:                             2122.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       6059.8110   1068.427      5.672      0.0

In [26]:
display(wind_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION,MARGINAL_PRICE,TECHNOLOGY,RENEWABLE
14,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01BALWIND_G01,G,10.0,3596.1318,1.0001,3596.4914,-0.3596,0.0,3536.43,Wind,Renewable
32,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01CAPRIS_G01,G,13.0,3568.4388,1.0079,3596.4914,-28.0526,0.0,3536.43,Wind,Renewable
33,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01CAPRIS_G02,G,9.0,3568.4388,1.0079,3596.4914,-28.0526,0.0,3536.43,Wind,Renewable
74,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01NWIND_G01,G,1.7,3621.3072,0.9932,3596.4914,24.8158,0.0,3536.43,Wind,Renewable
75,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,01NWIND_G02,G,3.5,3621.3072,0.9932,3596.4914,24.8158,0.0,3536.43,Wind,Renewable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3619,2025-07-16 14:55,RTD,2025-07-16 15:00,CLUZ,01CAPRIS_G01,G,6.0,4465.2921,0.9896,4418.8937,46.3984,0.0,4361.89,Wind,Renewable
3620,2025-07-16 14:55,RTD,2025-07-16 15:00,CLUZ,01CAPRIS_G02,G,9.0,4465.2921,0.9896,4418.8937,46.3984,0.0,4361.89,Wind,Renewable
3661,2025-07-16 14:55,RTD,2025-07-16 15:00,CLUZ,01NWIND_G01,G,1.3,4511.2486,0.9796,4418.8937,92.3549,0.0,4361.89,Wind,Renewable
3662,2025-07-16 14:55,RTD,2025-07-16 15:00,CLUZ,01NWIND_G02,G,4.0,4511.2486,0.9796,4418.8937,92.3549,0.0,4361.89,Wind,Renewable


In [27]:
# Filter for Geothermal technology
geothermal_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Geothermal'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(geothermal_df) >= 2:
    X = geothermal_df['SCHED_MW']
    y = geothermal_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    geothermal_model = sm.OLS(y, X).fit()
    print(geothermal_model.summary())
else:
    print("Not enough data for Geothermal regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.177
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     58.94
Date:                Tue, 09 Sep 2025   Prob (F-statistic):           2.89e-13
Time:                        14:16:23   Log-Likelihood:                -2744.6
No. Observations:                 276   AIC:                             5493.
Df Residuals:                     274   BIC:                             5500.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5604.4791    391.171     14.327      0.0

In [28]:
# Regression analysis for Gas only: MARGINAL_PRICE ~ SCHED_MW
# Filter for Gas technology
gas_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Gas'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(gas_df) >= 2:
    X = gas_df['SCHED_MW']
    y = gas_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    gas_model = sm.OLS(y, X).fit()
    print(gas_model.summary())
else:
    print("Not enough data for Gas regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                   0.04983
Date:                Tue, 09 Sep 2025   Prob (F-statistic):              0.824
Time:                        14:16:23   Log-Likelihood:                -1081.0
No. Observations:                 144   AIC:                             2166.
Df Residuals:                     142   BIC:                             2172.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4744.2970    128.430     36.941      0.0

In [29]:
# Regression analysis for Biomass only: MARGINAL_PRICE ~ SCHED_MW

# Filter for Biomass technology
biomass_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Biomass'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(biomass_df) >= 2:
    X = biomass_df['SCHED_MW']
    y = biomass_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    biomass_model = sm.OLS(y, X).fit()
    print(biomass_model.summary())
else:
    print("Not enough data for Biomass regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.331
Model:                            OLS   Adj. R-squared:                  0.328
Method:                 Least Squares   F-statistic:                     109.1
Date:                Tue, 09 Sep 2025   Prob (F-statistic):           5.34e-21
Time:                        14:16:23   Log-Likelihood:                -2105.3
No. Observations:                 222   AIC:                             4215.
Df Residuals:                     220   BIC:                             4221.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2300.4034    398.473      5.773      0.0

In [30]:
# Regression analysis for Diesel only: MARGINAL_PRICE ~ SCHED_MW

# Filter for Diesel technology
diesel_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Diesel'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(diesel_df) >= 2:
    X = diesel_df['SCHED_MW']
    y = diesel_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    diesel_model = sm.OLS(y, X).fit()
    print(diesel_model.summary())
else:
    print("Not enough data for Diesel regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.557
Model:                            OLS   Adj. R-squared:                  0.554
Method:                 Least Squares   F-statistic:                     208.4
Date:                Tue, 09 Sep 2025   Prob (F-statistic):           4.01e-31
Time:                        14:16:23   Log-Likelihood:                -1643.1
No. Observations:                 168   AIC:                             3290.
Df Residuals:                     166   BIC:                             3297.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.872e+04    511.556     36.599      0.0

In [31]:
display(diesel_df)

,RUN_TIME,MKT_TYPE,TIME_INTERVAL,REGION_NAME,RESOURCE_NAME,RESOURCE_TYPE,SCHED_MW,LMP,LOSS_FACTOR,LMP_SMP,LMP_LOSS,LMP_CONGESTION,MARGINAL_PRICE,TECHNOLOGY,RENEWABLE
154,2025-07-16 14:00,RTD,2025-07-16 14:05,CLUZ,03RCBMI_G02,G,5.50,3569.1581,1.0077,3596.4914,-27.3333,0.0,3536.4300,Diesel,Non-Renewable
304,2025-07-16 14:00,RTD,2025-07-16 14:05,CMIN,12TM2_U02,G,17.00,3907.2734,0.9221,3602.8339,304.4395,0.0,3458.0000,Diesel,Non-Renewable
305,2025-07-16 14:00,RTD,2025-07-16 14:05,CMIN,12TM2_U02,G,17.00,3907.2734,0.9221,3602.8339,304.4395,0.0,3458.0000,Diesel,Non-Renewable
306,2025-07-16 14:00,RTD,2025-07-16 14:05,CMIN,12TM2_U02,G,17.00,3907.2734,0.9221,3602.8339,304.4395,0.0,3458.0000,Diesel,Non-Renewable
307,2025-07-16 14:00,RTD,2025-07-16 14:05,CMIN,12TM2_U02,G,17.00,3907.2734,0.9221,3602.8339,304.4395,0.0,3458.0000,Diesel,Non-Renewable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3514,2025-07-16 14:50,RTD,2025-07-16 14:55,CLUZ,03RCBMI_G02,G,5.50,4387.0802,1.0073,4419.3414,-32.2612,0.0,4361.8900,Diesel,Non-Renewable
3574,2025-07-16 14:50,RTD,2025-07-16 14:55,CMIN,13TM1_U02,G,22.04,3986.1581,0.9020,3595.3442,390.8139,0.0,3458.0021,Diesel,Non-Renewable
3740,2025-07-16 14:55,RTD,2025-07-16 15:00,CLUZ,03RCBMI_G02,G,5.50,4386.1939,1.0074,4418.8937,-32.6998,0.0,4361.8900,Diesel,Non-Renewable
3841,2025-07-16 14:55,RTD,2025-07-16 15:00,CMIN,13TM1_U02,G,21.23,3987.5144,0.9009,3592.3553,395.1591,0.0,3458.0012,Diesel,Non-Renewable


In [32]:
# Regression analysis for Battery only: MARGINAL_PRICE ~ SCHED_MW

# Filter for Battery technology
battery_df = filtered_df[filtered_df['TECHNOLOGY'] == 'Battery'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(battery_df) >= 2:
    X = battery_df['SCHED_MW']
    y = battery_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    battery_model = sm.OLS(y, X).fit()
    print(battery_model.summary())
else:
    print("Not enough data for Battery regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 09 Sep 2025   Prob (F-statistic):                nan
Time:                        14:16:24   Log-Likelihood:                -172.65
No. Observations:                  17   AIC:                             347.3
Df Residuals:                      16   BIC:                             348.1
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SCHED_MW    1270.4264    155.678      8.161      0.0

c:\Users\theam\anaconda3\envs\virtual\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: UserWarning: `kurtosistest` p-value may be inaccurate with fewer than 20 observations; only n=17 observations were given.
  return hypotest_fun_in(*args, **kwds)


In [33]:
# Regression analysis for BioGas only: MARGINAL_PRICE ~ SCHED_MW

# Filter for BioGas technology
biogas_df = filtered_df[filtered_df['TECHNOLOGY'] == 'BioGas'].dropna(subset=['MARGINAL_PRICE', 'SCHED_MW'])

if len(biogas_df) >= 2:
    X = biogas_df['SCHED_MW']
    y = biogas_df['MARGINAL_PRICE']
    X = sm.add_constant(X)  # Adds intercept
    biogas_model = sm.OLS(y, X).fit()
    print(biogas_model.summary())
else:
    print("Not enough data for BioGas regression.")

                            OLS Regression Results                            
Dep. Variable:         MARGINAL_PRICE   R-squared:                      -0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 09 Sep 2025   Prob (F-statistic):                nan
Time:                        14:16:24   Log-Likelihood:                -189.06
No. Observations:                  22   AIC:                             380.1
Df Residuals:                      21   BIC:                             381.2
Df Model:                           0                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
SCHED_MW    2764.9353    203.525     13.585      0.0